In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
from peft import PeftModel, PeftConfig
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/home/ysx/miniconda3/envs/llama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
peft_model_id = "/home/ysx/src/AI/llm_demo/outputs/checkpoint-3000"
config = PeftConfig.from_pretrained(peft_model_id)


In [3]:
inference_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards: 100%|██████████| 8/8 [00:13<00:00,  1.71s/it]


In [4]:
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path,
    trust_remote_code=True
)

In [5]:
model = PeftModel.from_pretrained(inference_model, peft_model_id, device_map="auto")

In [2]:
tokenizer = AutoTokenizer.from_pretrained(
    "/home/ysx/models/internlm-chat-7b",
    trust_remote_code=True
)

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "/home/ysx/models/internlm-chat-7b",
    load_in_4bit=True,
    device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards: 100%|██████████| 8/8 [00:24<00:00,  3.11s/it]


In [6]:
model = model.eval()


In [7]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): InternLMForCausalLM(
      (model): InternLMModel(
        (embed_tokens): Embedding(103168, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x InternLMDecoderLayer(
            (self_attn): InternLMAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                

In [8]:
tokenizer

InternLMTokenizer(name_or_path='/home/ysx/models/internlm-chat-7b', vocab_size=103168, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [9]:
def generate_prompt(instruction, input):
    return (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        f"### instruction:\n{instruction}\n\n### input:\n{input}\n\n### responds:\n"
    )

In [10]:
text = generate_prompt("请回答下面问题", "五行是什么")

In [19]:
text = "样本A:乙方有权在新作游戏开发后的角色设定美术设定、玩法设定、Demo产出、游戏上线等多个时间节点对研发内容进行审核，甲方应主动在相关时间点以邮件方式通知乙方参与版本审核，并在邮件中通知相关的审核流程。请将样本A中的“新作游戏开发后的角色设定美术设定、玩法设定、Demo产出、游戏上线等多个时间节点”修改为“新作游戏开发过程中在角色设定、美术设定、玩法设定、Demo产出、游戏上线等多个时间节点”。\n请将样本A中的“甲方应主动在相关时间点以邮件方式通知乙方参与版本审核”修改为“甲方应提前在相关时间点以邮件方式通知乙方参与版本审核”。请在样本A中增加以下内容：“若甲乙双方均为在约定时间履行告知义务,所出现的损失由各方承担”。"

In [11]:
response, history = model.chat(tokenizer, text, history=[])
print(response)


/home/ysx/miniconda3/envs/llama/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


五行是指自然界中的五种元素，通常用木、火、土、金、水来表示。这些元素存在于自然界的万物中，并影响着它们的生长、发展、病邪、治疗和保健。五行学说是中医理论中描述自然界和人体结构及功能的一种理论。


In [5]:
response, history = model.chat(tokenizer, "脏腑病机是什么", history=[])
print(response)

脏腑病机是中医学理论体系中一个重要的概念，它主要涉及到人体五脏六腑的生理功能和病理变化。脏腑病机主要包括了脏腑功能失调、气血津液运行失常、气机郁滞等，这些因素相互作用，导致了脏腑的病理变化。中医学认为，脏腑病机是中医诊断和治疗疾病的重要依据，通过辨证论治，可以有效地调整脏腑功能，促进人体健康。
